# Unified Experiment Code
> Resistance is futile.

In [ ]:
# default_exp unified_experiment

In [ ]:
# hide
import blackhc.project.script

Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
import traceback
from dataclasses import dataclass
from typing import Optional, Type, Union

import torch
import torch.utils.data
from blackhc.project import is_run_from_ipython
from blackhc.project.experiment import embedded_experiments

import wandb
from batchbald_redux import acquisition_functions, baseline_acquisition_functions
from batchbald_redux.acquisition_functions import (
    CandidateBatchComputer,
    EvalDatasetBatchComputer,
    EvalModelBatchComputer,
)
from batchbald_redux.black_box_model_training import evaluate
from batchbald_redux.dataset_challenges import get_base_dataset_index, get_target
from batchbald_redux.di import DependencyInjection
from batchbald_redux.experiment_data import (
    ExperimentData,
    ExperimentDataConfig,
    OoDDatasetConfig,
    StandardExperimentDataConfig,
)
from batchbald_redux.experiment_logging import init_wandb, log2wandb
from batchbald_redux.models import MnistModelTrainer
from batchbald_redux.resnet_models import Cifar10ModelTrainer
from batchbald_redux.train_eval_model import (
    TrainEvalModel,
    TrainSelfDistillationEvalModel,
)
from batchbald_redux.trained_model import BayesianEnsembleModelTrainer, ModelTrainer

In [ ]:
# exports


@dataclass
class ActiveLearner:
    acquisition_size: int
    max_training_set: int

    num_validation_samples: int

    acquisition_function: Union[CandidateBatchComputer, EvalModelBatchComputer]
    train_eval_model: TrainEvalModel
    model_trainer: ModelTrainer
    data: ExperimentData

    disable_training_augmentations: bool

    device: Optional

    def __call__(self, log):
        log["seed"] = torch.seed()

        # Active Learning setup
        data = self.data

        train_augmentations = data.train_augmentations if not self.disable_training_augmentations else None

        model_trainer = self.model_trainer
        train_eval_model = self.train_eval_model

        train_loader = model_trainer.get_train_dataloader(data.active_learning.training_dataset)
        pool_loader = model_trainer.get_evaluation_dataloader(data.active_learning.pool_dataset)
        validation_loader = model_trainer.get_evaluation_dataloader(data.validation_dataset)
        test_loader = model_trainer.get_evaluation_dataloader(data.test_dataset)

        log["active_learning_steps"] = []
        active_learning_steps = log["active_learning_steps"]

        acquisition_function = self.acquisition_function

        num_iterations = 0
        max_iterations = int(
            1.5 * (self.max_training_set - len(data.active_learning.training_dataset)) / self.acquisition_size
        )

        # Active Training Loop
        while True:
            training_set_size = len(data.active_learning.training_dataset)
            print(f"Training set size {training_set_size}:")

            # iteration_log = dict(training={}, pool_training={}, evaluation_metrics=None, acquisition=None)
            active_learning_steps.append({})
            iteration_log = active_learning_steps[-1]

            iteration_log["training"] = {}

            # TODO: this is a hack! :(
            if data.ood_dataset is None:
                loss = validation_loss = torch.nn.NLLLoss()
            elif data.ood_exposure:
                loss = torch.nn.KLDivLoss(log_target=False, reduction="batchmean")
                validation_loss = torch.nn.NLLLoss()
            else:
                loss = validation_loss = torch.nn.NLLLoss()

            trained_model = model_trainer.get_trained(
                train_loader=train_loader,
                train_augmentations=train_augmentations,
                validation_loader=validation_loader,
                log=iteration_log["training"],
                wandb_key_path="model_training",
                loss=loss,
                validation_loss=validation_loss,
            )

            evaluation_metrics = evaluate(
                model=trained_model,
                num_samples=self.num_validation_samples,
                loader=test_loader,
                device=self.device,
                storage_device="cpu",
            )
            iteration_log["evaluation_metrics"] = evaluation_metrics
            log2wandb(evaluation_metrics, commit=False)
            print(f"Perf after training {evaluation_metrics}")

            if training_set_size >= self.max_training_set or num_iterations >= max_iterations:
                log2wandb({}, commit=True)
                print("Done.")                
                break

            if isinstance(acquisition_function, CandidateBatchComputer):
                candidate_batch = acquisition_function.compute_candidate_batch(trained_model, pool_loader, self.device)
            elif isinstance(acquisition_function, EvalDatasetBatchComputer):
                if len(data.evaluation_dataset) > 0:
                    eval_loader = model_trainer.get_evaluation_dataloader(data.evaluation_dataset)
                else:
                    eval_loader = pool_loader

                candidate_batch = acquisition_function.compute_candidate_batch(
                    model=trained_model, pool_loader=pool_loader, eval_loader=eval_loader, device=self.device
                )
            elif isinstance(acquisition_function, EvalModelBatchComputer):
                if len(data.evaluation_dataset) > 0:
                    eval_dataset = data.evaluation_dataset
                else:
                    eval_dataset = data.active_learning.pool_dataset

                iteration_log["eval_training"] = {}
                trained_eval_model = train_eval_model(
                    model_trainer=model_trainer,
                    training_dataset=data.active_learning.training_dataset,
                    train_augmentations=train_augmentations,
                    eval_dataset=eval_dataset,
                    validation_loader=validation_loader,
                    trained_model=trained_model,
                    storage_device=data.device,
                    device=self.device,
                    training_log=iteration_log["eval_training"],
                    wandb_key_path="eval_model_training",
                )

                candidate_batch = acquisition_function.compute_candidate_batch(
                    trained_model, trained_eval_model, pool_loader, device=self.device
                )
            else:
                raise ValueError(f"Unknown acquisition function {acquisition_function}!")

            candidate_global_dataset_indices = []
            candidate_labels = []
            candidate_images = []
            for index in candidate_batch.indices:
                base_di = get_base_dataset_index(data.active_learning.pool_dataset, index)
                dataset_type = "ood" if base_di.dataset == data.ood_dataset else "id"
                candidate_global_dataset_indices.append((dataset_type, base_di.index))
                if dataset_type == "id":
                    label = get_target(data.active_learning.pool_dataset, index).tolist()
                else:
                    label = None
                candidate_labels.append(label)
                candidate_images.append(wandb.Image(data.active_learning.pool_dataset[index][0]))

            acquisition_info = dict(
                indices=candidate_global_dataset_indices, labels=candidate_labels, scores=candidate_batch.scores
            )
            iteration_log["acquisition"] = acquisition_info
            
            acquistion_batch_table = wandb.Table(
                data=list(zip(*zip(*candidate_global_dataset_indices), candidate_images, candidate_labels, candidate_batch.scores)),
                columns=["dataset", "index", "sample", "label", "score"])    
            log2wandb(dict(acquisition=acquistion_batch_table), commit=False)

            print(candidate_batch)
            print(candidate_global_dataset_indices)

            if data.ood_dataset is None:
                data.active_learning.acquire(candidate_batch.indices)
            elif data.ood_exposure:
                data.active_learning.acquire(candidate_batch.indices)
            else:
                data.active_learning.acquire(
                    [index for index, label in zip(candidate_batch.indices, candidate_labels) if label != -1]
                )

            ls = ", ".join(f"{label} ({score:.4})" for label, score in zip(candidate_labels, candidate_batch.scores))
            print(f"Acquiring (label, score)s: {ls}")

            num_iterations += 1
            log2wandb({}, commit=True)
            
            


@dataclass
class UnifiedExperiment:
    seed: int

    experiment_data_config: ExperimentDataConfig

    acquisition_size: int = 5
    max_training_set: int = 200

    max_training_epochs: int = 300

    num_pool_samples: int = 100
    num_validation_samples: int = 20
    num_training_samples: int = 1

    device: str = "cuda"
    acquisition_function: Union[
        Type[CandidateBatchComputer], Type[EvalModelBatchComputer]
    ] = None  # acquisition_functions.BALD
    train_eval_model: Type[TrainEvalModel] = TrainSelfDistillationEvalModel
    model_trainer_factory: Type[ModelTrainer] = None  # Cifar10ModelTrainer
    ensemble_size: int = 1

    temperature: float = 1.0
    coldness: float = 1.0
    stochastic_mode: acquisition_functions.StochasticMode = None
    epig_bootstrap_type: acquisition_functions.BootstrapType = acquisition_functions.BootstrapType.NO_BOOTSTRAP
    epig_bootstrap_factor: float = 1.0
    epig_dtype: torch.dtype = torch.double
    disable_training_augmentations: bool = False
    cache_explicit_eval_model: bool = False
    resnet18_dropout_head: bool = True

    def load_experiment_data(self) -> ExperimentData:
        print(self.experiment_data_config)
        return self.experiment_data_config.load(self.device)

    # Simple Dependency Injection
    def create_acquisition_function(self):
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.acquisition_function)

    def create_train_eval_model(self) -> TrainEvalModel:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.train_eval_model)

    def create_model_trainer(self) -> ModelTrainer:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.model_trainer_factory)

    def run(self, store):
        init_wandb(self)

        torch.manual_seed(self.seed)

        # Active Learning setup
        data = self.load_experiment_data()
        store["dataset_info"] = dict(training=repr(data.active_learning.base_dataset), test=repr(data.test_dataset))
        store["initial_training_set_indices"] = data.initial_training_set_indices
        store["evaluation_set_indices"] = data.evaluation_set_indices
        
        wandb.config.initial_training_set_indices = data.initial_training_set_indices
        wandb.config.evaluation_set_indices = data.evaluation_set_indices
        wandb.config["dataset_info"] = store["dataset_info"]

        acquisition_function = self.create_acquisition_function()
        model_trainer = self.create_model_trainer()
        if self.ensemble_size > 1:
            model_trainer = BayesianEnsembleModelTrainer(model_trainer=model_trainer, ensemble_size=self.ensemble_size)
        train_eval_model = self.create_train_eval_model()

        active_learner = ActiveLearner(
            acquisition_size=self.acquisition_size,
            max_training_set=self.max_training_set,
            num_validation_samples=self.num_validation_samples,
            disable_training_augmentations=self.disable_training_augmentations,
            acquisition_function=acquisition_function,
            train_eval_model=train_eval_model,
            model_trainer=model_trainer,
            data=data,
            device=self.device,
        )

        active_learner(store)
        
        wandb.finish()

## CIFAR-10 vs SVHN Coverage

In [ ]:
# experiment
# OOD experiment (ood_exposure=True)

experiment = UnifiedExperiment(
    experiment_data_config=StandardExperimentDataConfig(
        id_dataset_name="CIFAR-10",
        id_repetitions=1,
        initial_training_set_size=20,
        validation_set_size=4096,
        validation_split_random_state=0,
        evaluation_set_size=100,
        add_dataset_noise=False,
        ood_dataset_config=OoDDatasetConfig(ood_dataset_name="SVHN", ood_repetitions=1, ood_exposure=True),
    ),
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 40,
    acquisition_function=acquisition_functions.EvalBALD,
    acquisition_size=10,
    num_pool_samples=2,
    device="cuda",
    model_trainer_factory=Cifar10ModelTrainer,
)

results = {}
experiment.run(results)
results

accuracy,▁
crossentropy,▁
accuracy,0.1347
crossentropy,2.82898


StandardExperimentDataConfig(id_dataset_name='CIFAR-10', id_repetitions=1, initial_training_set_size=20, validation_set_size=4096, validation_split_random_state=0, evaluation_set_size=100, add_dataset_noise=False, ood_dataset_config=OoDDatasetConfig(ood_dataset_name='SVHN', ood_repetitions=1, ood_exposure=True))
Files already downloaded and verified


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

Files already downloaded and verified
Using downloaded and verified file: data/SVHN/train_32x32.mat
Using downloaded and verified file: data/SVHN/test_32x32.mat
Creating: EvalBALD(
	acquisition_size=10,
	num_pool_samples=2
)
Creating: Cifar10ModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=1,
	resnet18_dropout_head=True
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=2
)
Training set size 20:
Cosine Annealing


100%|##########| 1/1 [00:00<?, ?it/s]

...

Epoch 1 metrics: {'accuracy': 0.139892578125, 'crossentropy': 2.311521530151367}
{'model_training/val_metrics': <wandb.data_types.Table object at 0x7f079a1d9e80>}


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.1407, 'crossentropy': tensor(2.3343), '_timestamp': 1648593941, '_runtime': 251}
Done.



accuracy,█▁▄▅▅
crossentropy,▄█▂▂▁
accuracy,0.1407
crossentropy,2.33432


{'dataset_info': {'training': "'CIFAR-10 (Train, seed=0, 45904 samples)' | one_hot_targets{'num_classes': 10} + 'SVHN (Train, seed=0, 73257 samples)' | uniform_targets{'num_classes': 10}",
  'test': "'CIFAR-10 (Test)'"},
 'initial_training_set_indices': [29863,
  22519,
  21079,
  37521,
  15583,
  23405,
  44362,
  35084,
  29380,
  26994,
  39909,
  29333,
  527,
  31668,
  43591,
  12293,
  39247,
  36879,
  7979,
  33280],
 'evaluation_set_indices': [4291,
  37709,
  11949,
  12149,
  17798,
  32245,
  235,
  38778,
  32864,
  39123,
  22797,
  43486,
  5813,
  40306,
  4570,
  15667,
  36937,
  25830,
  2404,
  30096,
  19730,
  11343,
  17034,
  43455,
  15770,
  7894,
  36848,
  11067,
  12968,
  29454,
  33432,
  21672,
  38040,
  259,
  29571,
  41639,
  43971,
  13338,
  42364,
  11109,
  18814,
  21474,
  10681,
  40169,
  9997,
  12784,
  22626,
  36949,
  33130,
  13688,
  6466,
  28263,
  45060,
  10422,
  7005,
  3143,
  26143,
  28581,
  13207,
  10038,
  6750,
  21221,

In [ ]:
# experiment
# OOD experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    experiment_data_config=StandardExperimentDataConfig(
        id_dataset_name="CIFAR-10",
        id_repetitions=1,
        initial_training_set_size=20,
        validation_set_size=4096,
        validation_split_random_state=0,
        evaluation_set_size=100,
        add_dataset_noise=False,
        ood_dataset_config=OoDDatasetConfig(ood_dataset_name="SVHN", ood_repetitions=1, ood_exposure=False),
    ),
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.EvalBALD,
    acquisition_size=10,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

## MNIST vs FashionMNIST Coverage

In [ ]:
# experiment
# MNIST OOD experiment (ood_exposure=True)

experiment = UnifiedExperiment(
    experiment_data_config=StandardExperimentDataConfig(
        id_dataset_name="MNIST",
        id_repetitions=1,
        initial_training_set_size=20,
        validation_set_size=4096,
        validation_split_random_state=0,
        evaluation_set_size=100,
        add_dataset_noise=False,
        ood_dataset_config=OoDDatasetConfig(ood_dataset_name="FashionMNIST", ood_repetitions=1, ood_exposure=True),
    ),
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.BALD,
    acquisition_size=10,
    model_trainer_factory=MnistModelTrainer,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

Creating: BALD(
	acquisition_size=10,
	num_pool_samples=2
)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=1
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=2
)
Training set size 20:


100%|##########| 1/1 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

...

[1/32]   3%|3          [00:00<?]

Epoch metrics: {'accuracy': 0.349853515625, 'crossentropy': 1.974664144217968}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.349853515625)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.3325, 'crossentropy': tensor(1.9364)}
Done.


{'dataset_info': {'training': "'MNIST (Train, seed=0, 55904 samples)' | one_hot_targets{'num_classes': 10} + 'FashionMNIST (Train, seed=0, 60000 samples)' | uniform_targets{'num_classes': 10}",
  'test': "'MNIST (Test)'"},
 'initial_training_set_indices': [47227,
  11511,
  18383,
  41080,
  32837,
  24393,
  23904,
  11784,
  20439,
  35043,
  27367,
  30426,
  32361,
  26116,
  24386,
  4689,
  44895,
  24211,
  17212,
  3478],
 'evaluation_set_indices': [49,
  12425,
  22159,
  22739,
  701,
  11289,
  4629,
  53972,
  52127,
  6277,
  18804,
  25364,
  18841,
  12957,
  10232,
  42477,
  35057,
  43089,
  48907,
  15149,
  24587,
  52160,
  19050,
  30188,
  50376,
  17140,
  26654,
  28683,
  36031,
  32477,
  32792,
  52660,
  52712,
  49702,
  40661,
  10190,
  18143,
  16617,
  54012,
  38376,
  1924,
  39518,
  1625,
  7896,
  1701,
  28972,
  3717,
  42815,
  6274,
  44154,
  44312,
  10944,
  14937,
  6358,
  1182,
  55213,
  9292,
  33572,
  13252,
  30667,
  31242,
  51852

## MNIST only

In [ ]:
# experiment
# MNIST experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    experiment_data_config=StandardExperimentDataConfig(
        id_dataset_name="MNIST",
        id_repetitions=1,
        initial_training_set_size=20,
        validation_set_size=4096,
        validation_split_random_state=0,
        evaluation_set_size=100,
        add_dataset_noise=False,
        ood_dataset_config=None,
    ),
    seed=1,
    max_training_epochs=5,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.BALD,
    acquisition_size=10,
    model_trainer_factory=MnistModelTrainer,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: BALD(
	acquisition_size=10,
	num_pool_samples=2
)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=5
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=2
)
Training set size 20:


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

 20%|##        | 1/5 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

...

[1/32]   3%|3          [00:00<?]

Epoch metrics: {'accuracy': 0.7548828125, 'crossentropy': 1.4579520002007484}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.7548828125)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.751, 'crossentropy': tensor(0.8279)}
Done.


{'dataset_info': {'training': "'MNIST (Train, seed=0, 55904 samples)'",
  'test': "'MNIST (Test)'"},
 'initial_training_set_indices': [47227,
  11511,
  18383,
  41080,
  32837,
  24393,
  23904,
  11784,
  20439,
  35043,
  27367,
  30426,
  32361,
  26116,
  24386,
  4689,
  44895,
  24211,
  17212,
  3478],
 'evaluation_set_indices': [49,
  12425,
  22159,
  22739,
  701,
  11289,
  4629,
  53972,
  52127,
  6277,
  18804,
  25364,
  18841,
  12957,
  10232,
  42477,
  35057,
  43089,
  48907,
  15149,
  24587,
  52160,
  19050,
  30188,
  50376,
  17140,
  26654,
  28683,
  36031,
  32477,
  32792,
  52660,
  52712,
  49702,
  40661,
  10190,
  18143,
  16617,
  54012,
  38376,
  1924,
  39518,
  1625,
  7896,
  1701,
  28972,
  3717,
  42815,
  6274,
  44154,
  44312,
  10944,
  14937,
  6358,
  1182,
  55213,
  9292,
  33572,
  13252,
  30667,
  31242,
  51852,
  33800,
  42491,
  31258,
  42348,
  42109,
  47515,
  1844,
  9314,
  18329,
  15263,
  52934,
  25225,
  25735,
  131

## MNIST only + BADGE

In [ ]:
# experiment
# MNIST experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    experiment_data_config=StandardExperimentDataConfig(
        id_dataset_name="MNIST",
        id_repetitions=1,
        initial_training_set_size=20,
        validation_set_size=4096,
        validation_split_random_state=0,
        evaluation_set_size=100,
        add_dataset_noise=False,
        ood_dataset_config=None,
    ),
    seed=1,
    max_training_epochs=5,
    max_training_set=20 + 10,
    acquisition_function=baseline_acquisition_functions.BADGE,
    acquisition_size=10,
    model_trainer_factory=MnistModelTrainer,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: BADGE(
	acquisition_size=10
)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=5
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=2
)
Training set size 20:


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

 20%|##        | 1/5 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

...

[1/32]   3%|3          [00:00<?]

Epoch metrics: {'accuracy': 0.69384765625, 'crossentropy': 1.828642327338457}
RestoringEarlyStopping: 1 / 20
RestoringEarlyStopping: Restoring best parameters. (Score: 0.715576171875)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.7307, 'crossentropy': tensor(0.8910)}
Done.


{'dataset_info': {'training': "'MNIST (Train, seed=0, 55904 samples)'",
  'test': "'MNIST (Test)'"},
 'initial_training_set_indices': [47227,
  11511,
  18383,
  41080,
  32837,
  24393,
  23904,
  11784,
  20439,
  35043,
  27367,
  30426,
  32361,
  26116,
  24386,
  4689,
  44895,
  24211,
  17212,
  3478],
 'evaluation_set_indices': [49,
  12425,
  22159,
  22739,
  701,
  11289,
  4629,
  53972,
  52127,
  6277,
  18804,
  25364,
  18841,
  12957,
  10232,
  42477,
  35057,
  43089,
  48907,
  15149,
  24587,
  52160,
  19050,
  30188,
  50376,
  17140,
  26654,
  28683,
  36031,
  32477,
  32792,
  52660,
  52712,
  49702,
  40661,
  10190,
  18143,
  16617,
  54012,
  38376,
  1924,
  39518,
  1625,
  7896,
  1701,
  28972,
  3717,
  42815,
  6274,
  44154,
  44312,
  10944,
  14937,
  6358,
  1182,
  55213,
  9292,
  33572,
  13252,
  30667,
  31242,
  51852,
  33800,
  42491,
  31258,
  42348,
  42109,
  47515,
  1844,
  9314,
  18329,
  15263,
  52934,
  25225,
  25735,
  131

## MNIST only + (naive) EPIG

In [ ]:
# experiment
# MNIST experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    experiment_data_config=StandardExperimentDataConfig(
        id_dataset_name="MNIST",
        id_repetitions=1,
        initial_training_set_size=20,
        validation_set_size=4096,
        validation_split_random_state=0,
        evaluation_set_size=0,
        add_dataset_noise=False,
        ood_dataset_config=None,
    ),
    seed=1,
    max_training_epochs=5,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.EPIG,
    acquisition_size=10,
    model_trainer_factory=MnistModelTrainer,
    num_pool_samples=100,
    device="cuda",
)

results = {}
experiment.run(results)
results

StandardExperimentDataConfig(id_dataset_name='MNIST', id_repetitions=1, initial_training_set_size=20, validation_set_size=4096, validation_split_random_state=0, evaluation_set_size=0, add_dataset_noise=False, ood_dataset_config=None)
Creating: EPIG(
	acquisition_size=10,
	num_pool_samples=100,
	epig_bootstrap_type=BootstrapType.NO_BOOTSTRAP,
	epig_bootstrap_factor=1.0,
	epig_dtype=torch.float64
)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=5
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=100
)
Training set size 20:


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

 20%|##        | 1/5 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

...

get_predictions_labels:   0%|          | 0/5588400 [00:00<?, ?it/s]

Entropy:   0%|          | 0/55884 [00:00<?, ?it/s]

Entropy:   0%|          | 0/55884 [00:00<?, ?it/s]

Evaluation Set:   0%|          | 0/55884 [00:00<?, ?it/s]

KeyboardInterrupt: 

## CIFAR-10 only

In [ ]:
# experiment
# CIFAR-10 experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    experiment_data_config=StandardExperimentDataConfig(
        id_dataset_name="CIFAR-10",
        id_repetitions=1,
        initial_training_set_size=20,
        validation_set_size=4096,
        validation_split_random_state=0,
        evaluation_set_size=100,
        add_dataset_noise=False,
        ood_dataset_config=None,
    ),
    seed=1,
    max_training_epochs=5,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.BALD,
    acquisition_size=10,
    model_trainer_factory=Cifar10ModelTrainer,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Files already downloaded and verified


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

Files already downloaded and verified
Creating: BALD(
	acquisition_size=10,
	num_pool_samples=2
)
Creating: Cifar10ModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=5
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=2
)
Training set size 20:
Cosine Annealing


 20%|##        | 1/5 [00:00<?, ?it/s]

[1/39]   3%|2          [00:00<?]

...

[1/39]   3%|2          [00:00<?]

Epoch 5 metrics: {'accuracy': 0.181640625, 'crossentropy': 5.63788229227066}


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.1767, 'crossentropy': tensor(4.1496)}
Done.


{'dataset_info': {'training': "'CIFAR-10 (Train, seed=0, 45904 samples)'",
  'test': "'CIFAR-10 (Test)'"},
 'initial_training_set_indices': [29863,
  22519,
  21079,
  37521,
  15583,
  23405,
  44362,
  35084,
  29380,
  26994,
  39909,
  29333,
  527,
  31668,
  43591,
  12293,
  39247,
  36879,
  7979,
  33280],
 'evaluation_set_indices': [4291,
  37709,
  11949,
  12149,
  17798,
  32245,
  235,
  38778,
  32864,
  39123,
  22797,
  43486,
  5813,
  40306,
  4570,
  15667,
  36937,
  25830,
  2404,
  30096,
  19730,
  11343,
  17034,
  43455,
  15770,
  7894,
  36848,
  11067,
  12968,
  29454,
  33432,
  21672,
  38040,
  259,
  29571,
  41639,
  43971,
  13338,
  42364,
  11109,
  18814,
  21474,
  10681,
  40169,
  9997,
  12784,
  22626,
  36949,
  33130,
  13688,
  6466,
  28263,
  45060,
  10422,
  7005,
  3143,
  26143,
  28581,
  13207,
  10038,
  6750,
  21221,
  9097,
  26989,
  26153,
  13087,
  28498,
  39840,
  26870,
  24298,
  24804,
  28038,
  17767,
  31439,
  1247

In [ ]:
# exports

configs = [
    UnifiedExperiment(
        seed=seed + 1234,
        experiment_data_config=StandardExperimentDataConfig(
            id_dataset_name=id_dataset_name,
            id_repetitions=1,
            initial_training_set_size=20,
            validation_set_size=4096,
            validation_split_random_state=0,
            evaluation_set_size=evaluation_set_size,
            add_dataset_noise=False,
            ood_dataset_config=OoDDatasetConfig(
                ood_dataset_name=ood_dataset_name, ood_repetitions=1, ood_exposure=ood_exposure
            ),
        ),
        acquisition_function=acquisition_function,
        acquisition_size=5,
        num_pool_samples=num_pool_samples,
    )
    for seed in range(3)
    for acquisition_function in [acquisition_functions.BatchEvalBALD, acquisition_functions.BatchBALD]
    for evaluation_set_size in [1024]
    for num_pool_samples in [100]
    for ood_exposure in [True, False]
    for id_dataset_name, ood_dataset_name in [("CIFAR-10", "SVHN"), ("SVHN", "CIFAR-10")]
]

if not is_run_from_ipython() and __name__ == "__main__":
    for job_id, store in embedded_experiments(__file__, len(configs)):
        config = configs[job_id]
        config.seed += job_id
        print(config)
        store["config"] = dataclasses.asdict(config)
        store["log"] = {}

        try:
            config.run(store=store)
        except Exception:
            store["exception"] = traceback.format_exc()
            raise

In [ ]:
len(configs)

24

In [ ]:
# slow
import prettyprinter

prettyprinter.install_extras(include={"dataclasses"})

prettyprinter.pprint(configs)

[
    OodExperiment(
        seed=1234,
        uniform_ood=True,
        id_dataset_name='CIFAR-10',
        ood_dataset_name='SVHN',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=True,
        id_dataset_name='SVHN',
        ood_dataset_name='CIFAR-10',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=False,
        id_dataset_name='CIFAR-10',
        ood_dataset_name='SVHN',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=False,
        id_dataset_name='SVHN',
        ood_dataset_name='CIFAR-10',
        num_pool_samples=100,
        # class
        acquisit